# Train

In [1]:
import torch
import torchvision

In [2]:
torch.cuda.empty_cache()
#device = "cpu"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Device: " + device)
print(f"Devices count: {torch.cuda.device_count()}")

Device: cuda:0
Devices count: 1


In [3]:
import pandas
import numpy
import pickle

In [4]:
from pathlib import Path

In [5]:
from misc.data import DetectorDataset, AnnotationlessDetectorDataset, SimpleClassifierDataset, SupplementaryDataset, concatenate_collate_fn, detection_results_to_annotations

In [6]:
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife.data import transforms as pw_trans
from PytorchWildlife import utils as pw_utils

In [7]:
classifier_model_name = "swin_v2_s"
classifier_weights_name = "Swin_V2_S_Weights"
classifier_weights_subname = "IMAGENET1K_V1"

## Data

In [8]:
data_path = Path("./data/test_data_minprirodi/")
images_path = data_path / "images"
annotation_path = data_path / "annotation.csv"

In [9]:
model_path = Path("./data/train_data_minprirodi/") / f"models/{classifier_weights_name}/{classifier_weights_subname}"

In [10]:
dataset = AnnotationlessDetectorDataset(images_path)

## Model

In [11]:
detection_model = pw_detection.MegaDetectorV6(device=device, pretrained=True)

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs


In [14]:
annotations_pred = pandas.read_csv(data_path / "annotations_pred.csv")

In [15]:
annotations_pred

,Name,Bbox,xyxy_normalized_coords,label,confidence
0,3000160.jpg,"0.06437474489212036, 0.6150065660476685, 0.032...",[ 0.047978 0.60507 0.080771 0.62494],0,0.215246
1,3000723.jpg,"0.5633390545845032, 0.6898795366287231, 0.8733...",[ 0.12668 0.43802 1 0.94173],0,0.952901
2,3001267.jpg,"0.9469287991523743, 0.5806889533996582, 0.1046...",[ 0.89462 0.55519 0.99924 0.60619],0,0.499477
3,3001689.jpg,"0.5274641513824463, 0.6682726740837097, 0.6450...",[ 0.20495 0.49549 0.84998 0.84105],0,0.921705
4,3003898.jpg,"0.3841886520385742, 0.604388952255249, 0.21015...",[ 0.27911 0.48503 0.48926 0.72375],0,0.945228
...,...,...,...,...,...
1542,3999483.jpg,"0.315591037273407, 0.6522321105003357, 0.39442...",[ 0.11838 0.30512 0.5128 0.99934],0,0.972877
1543,3999817.jpg,"0.5, 0.7559560537338257, 1.0, 0.3814053535461426",[ 0 0.56525 1 0.94666],0,0.593780
1544,3999887.jpg,"0.435708224773407, 0.7554147243499756, 0.59468...",[ 0.13837 0.67448 0.73305 0.83635],0,0.831750
1545,3999887.jpg,"0.047514598816633224, 0.6751159429550171, 0.09...",[ 0 0.58704 0.095029 0.7632],0,0.602150


In [16]:
annotations_pred["Class"] = numpy.zeros(len(annotations_pred))
#annotations_pred["Class"] = annotation["Class"]

In [17]:
dataset = SimpleClassifierDataset(
    images_path,
    annotations_pred,
    torchvision.transforms.Resize((232, 232), interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
)

In [18]:
batch_size = 128

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [20]:
model = torch.load(model_path / "e8_0.98.pt")
#model = torch.load(model_path / "e17_1.00.pt")

In [21]:
_, y_pred = model.predict(dataloader, device)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [02:58<00:00, 13.74s/it]


In [22]:
test_annotations_pred = annotations_pred.copy()
test_annotations_pred["Class"] = 1 * (numpy.exp(y_pred[:,1]) > 0.3)

In [23]:
test_annotations_pred

,Name,Bbox,xyxy_normalized_coords,label,confidence,Class
0,3000160.jpg,"0.06437474489212036, 0.6150065660476685, 0.032...",[ 0.047978 0.60507 0.080771 0.62494],0,0.215246,0
1,3000723.jpg,"0.5633390545845032, 0.6898795366287231, 0.8733...",[ 0.12668 0.43802 1 0.94173],0,0.952901,1
2,3001267.jpg,"0.9469287991523743, 0.5806889533996582, 0.1046...",[ 0.89462 0.55519 0.99924 0.60619],0,0.499477,0
3,3001689.jpg,"0.5274641513824463, 0.6682726740837097, 0.6450...",[ 0.20495 0.49549 0.84998 0.84105],0,0.921705,0
4,3003898.jpg,"0.3841886520385742, 0.604388952255249, 0.21015...",[ 0.27911 0.48503 0.48926 0.72375],0,0.945228,1
...,...,...,...,...,...,...
1542,3999483.jpg,"0.315591037273407, 0.6522321105003357, 0.39442...",[ 0.11838 0.30512 0.5128 0.99934],0,0.972877,1
1543,3999817.jpg,"0.5, 0.7559560537338257, 1.0, 0.3814053535461426",[ 0 0.56525 1 0.94666],0,0.593780,0
1544,3999887.jpg,"0.435708224773407, 0.7554147243499756, 0.59468...",[ 0.13837 0.67448 0.73305 0.83635],0,0.831750,0
1545,3999887.jpg,"0.047514598816633224, 0.6751159429550171, 0.09...",[ 0 0.58704 0.095029 0.7632],0,0.602150,0


In [24]:
from datetime import datetime

In [25]:
test_annotations_pred[["Name", "Bbox", "Class"]].to_csv(data_path / f"submission_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S.%f')}.csv", index=False)